# Use `Lale` `AIF360` `DisparateImpactRemover` to  mitigate bias for credit risk AutoAI model

This notebook contains the steps and code to demonstrate support of AutoAI experiments in watsonx.ai service. It introduces commands for bias detecting and mitigation performed with `lale.lib.aif360` module. 

Some familiarity with Python is helpful. This notebook uses Python 3.12.

**NOTE:** The notebook is a continuation for sample notebook: <a href="https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/experiments/autoai/fairness/Use%20AutoAI%20to%20train%20fair%20models.ipynb" target="_blank" rel="noopener no referrer">"Use AutoAI to train fair models"</a>. 

## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Load historical experiment](#definition)
3. [Pipeline bias detection and mitigation](#bias)
5. [Deployment and score](#scoring)
6. [Clean up](#cleanup)
7. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U "autoai-libs==3.0.3" | tail -n 1
%pip install -U "lale[fairness]" | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

Enter your watsonx.ai password and hit enter:  ········


#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below


In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.


In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set the **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="definition"></a>
## 2. Load historical experiment

Initialize AutoAI experiment with watsonx.ai credentials and space.  

In [7]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, space_id=space_id)

List all previous AutoAI experiment runs named `'Credit Risk Prediction and bias detection - AutoAI'` which was run in sample notebook <a href="https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/experiments/autoai/fairness/Use%20AutoAI%20to%20train%20fair%20models.ipynb" target="_blank" rel="noopener no referrer">"Use AutoAI to train fair models"</a>. 

**NOTE:** If you don't have any experiment listed below please run the <a href="https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/experiments/autoai/fairness/Use%20AutoAI%20to%20train%20fair%20models.ipynb" target="_blank" rel="noopener no referrer">"Use AutoAI to train fair models"</a> notebook first and then continue running the current notebook. 

In [8]:
autoai_experiment_name = "Credit Risk Prediction and bias detection - AutoAI"

historical_experiments = experiment.runs(filter=autoai_experiment_name).list()
historical_experiments

,timestamp,run_id,state,auto_pipeline_optimizer name
0,2025-05-23T13:44:57.102Z,b7ee8857-8805-4d25-8456-b7aa8a366957,completed,Credit Risk Prediction and bias detection - Au...
1,2025-05-23T12:44:26.982Z,a3ce47c9-3da6-481f-9a71-59775dae5ecc,completed,Credit Risk Prediction and bias detection - Au...
2,2025-05-23T12:37:45.995Z,6f823c4e-17ff-4e59-96da-8a9d363db433,completed,Credit Risk Prediction and bias detection - Au...
3,2025-05-23T11:59:18.516Z,5ab6871d-e40b-4204-aa64-81611f60f36d,completed,Credit Risk Prediction and bias detection - Au...


Load last experiment run to variable `pipeline_optimizer`.

In [9]:
run_id = historical_experiments.run_id[0]

pipeline_optimizer = experiment.runs.get_optimizer(run_id)

In [10]:
summary = pipeline_optimizer.summary()
summary

,Enhancements,Estimator,training_accuracy_and_disparate_impact_(optimized),training_disparate_impact_Sex,training_roc_auc,holdout_disparate_impact_Sex,holdout_average_precision,holdout_log_loss,holdout_roc_auc,training_disparate_impact,...,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1,training_accuracy,holdout_disparate_impact,training_balanced_accuracy,holdout_disparate_impact_Age,training_f1,training_disparate_impact_Age
Pipeline Name,,,,,,,,,,,,,,,,,,,,,
Pipeline_1,,XGBClassifier,0.676887,1.009581,0.846120,1.046499,0.480936,0.419151,0.855620,1.825512,...,0.811623,0.754275,0.894970,0.867606,0.796567,1.431694,0.748223,1.426056,0.853965,2.329145
Pipeline_2,HPO,XGBClassifier,0.676887,1.009581,0.846120,1.046499,0.480936,0.419151,0.855620,1.825512,...,0.811623,0.754275,0.894970,0.867606,0.796567,1.431694,0.748223,1.426056,0.853965,2.329145
Pipeline_3,"HPO, FE",XGBClassifier,0.681126,1.009843,0.846576,1.057825,0.481095,0.416944,0.855187,1.787199,...,0.809619,0.755745,0.892283,0.865248,0.795004,1.463687,0.747200,1.451613,0.852646,2.286728
Pipeline_4,"HPO, FE, HPO",XGBClassifier,0.681126,1.009843,0.846576,1.057825,0.481095,0.416944,0.855187,1.787199,...,0.809619,0.755745,0.892283,0.865248,0.795004,1.463687,0.747200,1.451613,0.852646,2.286728
Pipeline_5,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(XGBClassifier),0.681126,1.009843,0.846576,1.057825,0.481095,0.416944,0.855187,1.787199,...,0.809619,0.755745,0.892283,0.865248,0.795004,1.463687,0.747200,1.451613,0.852646,2.286728


### Get selected pipeline model

Download pipeline model object from the AutoAI training job.

In [11]:
best_pipeline = pipeline_optimizer.get_pipeline()

### Get Credit Risk dataset from experiment configuration. 

In [12]:
data_connections = pipeline_optimizer.get_data_connections()

X_train, X_holdout, y_train, y_holdout = data_connections[0].read(
    with_holdout_split=True
)

X_holdout.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,CurrentResidenceDuration,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker
13,0_to_200,29.0,credits_paid_to_date,furniture,3705.0,less_100,less_1,3.0,female,co-applicant,3.0,car_other,44.0,none,own,1.0,skilled,1.0,none,yes
31,no_checking,26.0,prior_payments_delayed,appliances,4391.0,500_to_1000,greater_7,4.0,female,none,3.0,car_other,39.0,stores,own,2.0,unskilled,2.0,yes,yes
3673,less_0,20.0,credits_paid_to_date,retraining,4683.0,less_100,4_to_7,2.0,male,none,1.0,savings_insurance,46.0,stores,free,1.0,skilled,1.0,yes,yes
3611,no_checking,25.0,prior_payments_delayed,car_used,5426.0,500_to_1000,4_to_7,4.0,male,none,3.0,car_other,41.0,none,own,1.0,skilled,1.0,none,yes
433,greater_200,34.0,prior_payments_delayed,furniture,5470.0,100_to_500,4_to_7,4.0,male,none,4.0,car_other,33.0,none,own,2.0,skilled,1.0,none,no


<a id="bias"></a>
## 3. Bias detection and mitigation

The `fairness_info` dictionary contains some fairness-related metadata. The favorable and unfavorable label are values of the target class column that indicate whether the loan was granted or denied. A protected attribute is a feature that partitions the population into groups whose outcome should have parity. The credit-risk dataset has two protected attribute columns, sex and age. Each prottected attributes has monitored and reference group.


In [13]:
fairness_info = pipeline_optimizer.get_params()["fairness_info"]
fairness_info

{'favorable_labels': ['No Risk'],
 'protected_attributes': [{'feature': 'Sex',
   'monitored_group': ['female'],
   'reference_group': ['male']},
  {'feature': 'Age',
   'monitored_group': [[18, 25]],
   'reference_group': [[26, 75]]}],
 'unfavorable_labels': ['Risk']}

### Calculate fairness metrics

We will calculate some model metrics. Accuracy describes how accurate is the model according to dataset. 
Disparate impact is defined by comparing outcomes between a privileged group and an unprivileged group, 
so it needs to check the protected attribute to determine group membership for the sample record at hand. The closer to 1 is the value of disparate impact the less biased is the model. 
The third calculated metric takes the disparate impact into account along with accuracy. The best value of the score is 1.0.

In [14]:
import sklearn.metrics
from lale.lib.aif360 import disparate_impact, accuracy_and_disparate_impact

accuracy_scorer = sklearn.metrics.make_scorer(sklearn.metrics.accuracy_score)
print(f"accuracy {accuracy_scorer(best_pipeline, X_holdout, y_holdout):.1%}")

disparate_impact_scorer = disparate_impact(**fairness_info)
print(
    f"disparate impact {disparate_impact_scorer(best_pipeline, X_holdout, y_holdout):.2f}"
)

combined_scorer = accuracy_and_disparate_impact(**fairness_info)
print(
    f"accuracy and disparate impact metric {combined_scorer(best_pipeline, X_holdout, y_holdout):.2f}"
)

accuracy 81.2%
disparate impact 1.43
accuracy and disparate impact metric 0.76


## Refinery with lale

In this section we will use `DisparateImpactRemover` algorithm for mitigating fairness problems from `lale.lib.aif360` module. It modifies the features that are not the protected attribute in such a way that it is hard to predict the protected attribute from them. This algorithm has a hyperparameter `repair_level` that we will tune with hyperparameter optimization.

In [15]:
from lale.lib.aif360 import DisparateImpactRemover
from lale.pretty_print import ipython_display

ipython_display(DisparateImpactRemover.hyperparam_schema("repair_level"))

```python
{
    "description": "Repair amount from 0 = none to 1 = full.",
    "type": "number",
    "minimum": 0,
    "maximum": 1,
    "default": 1,
}
```

### Pipeline decomposition and new definition

Start by removing the last step of the pipeline, i.e., the final estimator.

In [16]:
prefix = best_pipeline.remove_last().freeze_trainable()
prefix.export_to_sklearn_pipeline()

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140371756645552',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                                                                 ('numpyreplacemissingvalues',
                                                                  NumpyReplaceMissingValues(missing_values=[])),
                                                                 ('numimputer',
                                                                  NumImputer(missing_values=nan,
                                                                             strategy='median')),
                                                                 ('optstandardscaler',
                                                                  OptStandardScaler(use_scaler_flag=False)),
                                                                 ('float32_transform',
                                                                  float32_transform())]))])),
                ('numpypermutearray',
                 NumpyPermuteArray(axis=0,
                                   permutation_indices=[0, 1, 2, 3, 5, 6, 7, 8,
                                                        9, 10, 11,

Initialize the `DisparateImpactRemover` with fairness configuration and pipeline without final estimator and add a new final step, which consists of a choice of two estimators. In this code, `|` is the or combinator (algorithmic choice). It defines a search space for another optimizer run.

In [17]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
from lale.operator_wrapper import wrap_imported_operators

wrap_imported_operators()

di_remover = DisparateImpactRemover(**fairness_info, preparation=prefix)
planned_fairer = di_remover >> (LR | RF)

Fairness metrics can be more unstable than accuracy, because they depend not just on the distribution of labels, but also on the distribution of privileged and unprivileged groups as defined by the protected attributes. In AI Automation, k-fold cross validation helps reduce overfitting. To get more stable results, we will stratify these k folds by both labels and groups with `FairStratifiedKFold` class.

In [18]:
from lale.lib.aif360 import FairStratifiedKFold

fair_cv = FairStratifiedKFold(**fairness_info, n_splits=3)

### Pipeline training

To automatically select the algorithm and tune its hyperparameters we use `auto_configure` method of lale pipeline. The combined metric `accuracy_and_disparate_impact` is used as scoring metric in evaluation process.   

In [19]:
from lale.lib.lale import Hyperopt

trained_fairer = planned_fairer.auto_configure(
    X_train,
    y_train,
    optimizer=Hyperopt,
    cv=fair_cv,
    max_evals=10,
    scoring=combined_scorer,
    best_score=1.0,
)

100%|██████████| 10/10 [00:26<00:00,  2.70s/trial, best loss: 0.1676332233430191]


#### Results 

Visualize the final pipeline and calculate its metrics.  

In [20]:
trained_fairer.export_to_sklearn_pipeline()

Pipeline(steps=[('_disparateimpactremoverimpl',
                 <lale.lib.aif360.disparate_impact_remover._DisparateImpactRemoverImpl object at 0x7faad87b7ad0>),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy',
                                        min_samples_leaf=0.3551854079747729,
                                        min_samples_split=0.3003021725296457,
                                        n_estimators=29))])

In [21]:
print(f"accuracy {accuracy_scorer(trained_fairer, X_holdout, y_holdout):.1%}")
print(
    f"disparate impact {disparate_impact_scorer(trained_fairer, X_holdout, y_holdout):.2f}"
)
print(
    f"accuracy and disparate impact metric {combined_scorer(trained_fairer, X_holdout, y_holdout):.2f}"
)

accuracy 66.5%
disparate impact 1.00
accuracy and disparate impact metric 0.83


**Summary:** As result of the described steps we received unbiased pipeline model based on disparate impact value, however the accuracy of the model decreased from 70% to 66%.

<a id="scoring"></a>
## 5. Deploy and Score
In this section you will learn how to deploy and score Lale pipeline model using WML instance.

#### Custom software specification

Created model is AutoAI model refined with Lale. We will create new software specification based on default Python 3.12 environment extended by  `autoai-libs` package.

In [22]:
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")
print("ID of default Python 3.12 software specification is: ", base_sw_spec_id)

ID of default Python 3.12 software specification is:  f47ae1c3-198e-5718-b59d-2ea471561e9e


Create the file defining dependencies of package extension.

In [23]:
with open("requirements.txt", "w") as file:
    file.write("autoai-libs")

The `requirements.txt` file describes details of package extention. Now you need to store new package extention with `APIClient`.

In [24]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "Scikit with autoai-libs",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Package extension for autoai-libs",
    client.package_extensions.ConfigurationMetaNames.TYPE: "requirements_txt",
}

pkg_extn_details = client.package_extensions.store(
    meta_props=meta_prop_pkg_extn, file_path="requirements.txt"
)
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


Create new software specification and add created package extention to it. 

In [25]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "Mitigated AutoAI bases on scikit specification",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for scikit with autoai-libs",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_sw_spec_id
    },
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)

status = client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


You can get details of created software specification using `client.software_specifications.get_details(sw_spec_id)`

### Store the model

In [26]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Fairer AutoAI model",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.6",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}
feature_vector = list(X_train.columns)

In [27]:
published_model = client.repository.store_model(
    model=best_pipeline.export_to_sklearn_pipeline(),
    meta_props=model_props,
    training_data=X_train.values,
    training_target=y_train.values,
    feature_names=feature_vector,
    label_column_names=["Risk"],
)

In [28]:
published_model_id = client.repository.get_model_id(published_model)

### Deployment creation

In [29]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of fairer model",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '99333010-9333-465f-ab57-bc354ca03efa' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='1e5d83bd-43c0-4c2b-95cf-dfd002687aed'
-----------------------------------------------------------------------------------------------




In [30]:
deployment_id = client.deployments.get_id(created_deployment)

#### Deployment scoring 

You need to pass scoring values as input data if the deployed model. Use `client.deployments.score()` method to get predictions from deployed model. 

In [31]:
values = X_holdout.values

scoring_payload = {"input_data": [{"values": values[:5]}]}

In [32]:
predictions = client.deployments.score(deployment_id, scoring_payload)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.7776124477386475, 0.22238758206367493]],
    ['No Risk', [0.7723912596702576, 0.22760875523090363]],
    ['No Risk', [0.7723912596702576, 0.22760875523090363]],
    ['No Risk', [0.7541248202323914, 0.24587517976760864]],
    ['No Risk', [0.7541248202323914, 0.24587517976760864]]]}]}

<a id="cleanup"></a>
## 5. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps

You successfully completed this notebook!

Check out the documentations of libraries used in this notebook:
- `ibm-watsonx-ai` [Online Documentation](https://ibm.github.io/watsonx-ai-python-sdk/index.html)
- `lale`: https://github.com/IBM/lale
- `aif360`: https://aif360.mybluemix.net/

### Authors

**Dorota Lączak**, Software Engineer at watsonx.ai.

Copyright © 2022-2025 IBM. This notebook and its source code are released under the terms of the MIT License.